In [1]:
import base64
import os
import json
from PyPDF2 import PdfReader
import numpy as np
import base64
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import uuid


In [2]:
class QueryEncoderPipeline:
    def __init__(self, model_name="sentence-transformers/multi-qa-mpnet-base-dot-v1"):
        self.model = SentenceTransformer(model_name)

    def __call__(self, query):
        return self.model.encode(query)

query_encoder_pipeline = QueryEncoderPipeline()

def ndarray_from_json(json):
    return np.frombuffer(base64.b64decode(json["bytes"]), dtype=json["type"]).reshape(json["shape"])

def query_embeddings_dict(query, embeddings_dict, top_k=10):
    query_embedding = query_encoder_pipeline(query)
    results = []
    for key, value in embeddings_dict.items():
        embedding = ndarray_from_json(value["embedding"])
        results.append((key, value["filename"], value["text"], np.dot(query_embedding, embedding)))
    return sorted(results, key=lambda x: x[3], reverse=True)[:top_k]



qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2", tokenizer="deepset/roberta-base-squad2")



In [4]:
with open("embeddings.json", 'r') as f:
    embeddings_dict = json.load(f)

query = "What is an infinite horizon MDP?"
results = query_embeddings_dict(query, embeddings_dict)

for result in results:

    context = result[2]
    answer = qa_model(question = query, context = context)

    print(f"{result[1]} - {result[3]}")
    print(f"\tAnswer: {answer['answer']}")
    print(f"\tContext: {context}")
    print()

data_directory/pdfs/Lecture_Notes_v1.0_687_F22.pdf - 30.684228897094727
	Answer: indefinite horizon
	Context: cause a transition tos∞after twenty seconds, the state must encode the current time step. 1. 7. 5 finite - horizon mdps thehorizon, l, of an mdp is the smallest integer such that [UNK], pr ( st = s∞ ) = 1. ( 26 ) ifl < ∞for all policies, then we say that the mdp is finite horizon. ifl = ∞ then the domain may be indefinite horizon orinfinite horizon. an mdp with indefinite horizon is one for which l = ∞, but where the agent will always enter s∞. one example of an indefinite

data_directory/pdfs/Lecture_Notes_v1.0_687_F22.pdf - 30.071758270263672
	Answer: an mdp where the agent may never enter s∞.
	Context: horizon mdp is one where the agent transitions to s∞with probability 0. 5 from every state. an infinite horizon mdp is an mdp where the agent may never enter s∞. for the cart - pole domain, how can we implement within pthat a transition tos∞must occur after 20 seconds? we achi